You can download and run this notebook locally, or you can run it for free in a cloud environment using Colab or Sagemaker Studio Lab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kirbyju/TCIA_Notebooks/blob/main/TCIA_Aspera_CLI_Downloads.ipynb)

[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_Aspera_CLI_Downloads.ipynb)

# Summary
Much of non-DICOM content in [The Cancer Imaging Archive (TCIA)](https://www.cancerimagingarchive.net/) is provided via links to IBM Aspera Faspex packages.  Aspera’s FASP protocol is designed to move data rapidly across networks with minimal disruption to other traffic.  Aspera’s Faspex application bundles data into packages that can be referenced via a web link (i.e., a URI).  When an Aspera Faspex link resolves in a browser, it presents a GUI that guides a user through the process of installing a browser extension and a local Aspera Connect client, if not already there, that moves the data using FASP between TCIA servers and the computer the browser is running on.
We frequently get requests from researchers for an option to download TCIA Faspex packages using a command line interface, suitable for inclusion in shell/batch scripts, that bypasses the GUI.  While not a part of the standard Aspera distributions, the IBM Aspera developers have provided an open source tool (Apache 2.0 license) called [ascli (aspera-cli)](https://github.com/IBM/aspera-cli) that allows a client to download an Aspera Faspex package using its URI.


# Install prerequisite software

In [ ]:
!apt install -y ruby ruby-dev rubygems ruby-json
!gem install aspera-cli
!ascli conf ascp install

# Download an Aspera package

Now that the prerequisite installations are complete, you will be able to use the ascli command to transfer a TCIA Faspex package using just its link by following these steps:
1.	Browse to the dataset landing page that describes the package,
2.	Find the “Download” button that one normally would use to download the package using the Faspex GUI, but do not click it,
3.	Right click that button icon and select “copy link address” or “copy link” or similar (depends on your browser) to extract the package URI into the clipboard or copy/paste buffer.

Replace the **url** parameter in the following cell with the link of the package you want to download.  

**Note:** If you don't change the package URL below it will only take a few seconds to download an example package (~100MB) from the [Cancer Moonshot Biobank - Gastroesophageal Cancer Collection (CMB-GEC)](https://doi.org/10.7937/E7KH-R486).


In [2]:
!ascli faspex5 packages receive \
  --url='https://faspex.cancerimagingarchive.net/aspera/faspex/public/package?context=eyJyZXNvdXJjZSI6InBhY2thZ2VzIiwidHlwZSI6ImV4dGVybmFsX2Rvd25sb2FkX3BhY2thZ2UiLCJpZCI6Ijc2MiIsInBhc3Njb2RlIjoiZDg0NmZlM2Q5ZjZjNzliYjUxYWU2MWMzNjJkNmE1ODJmMTc0YmVkYSIsInBhY2thZ2VfaWQiOiI3NjIiLCJlbWFpbCI6ImhlbHBAY2FuY2VyaW1hZ2luZ2FyY2hpdmUubmV0In0='

Receiving package 762
 Time: 00:00:03 =================================== 100% 274 Mbps Time: 00:00:03
+---------+---------+
| key     | value   |
+---------+---------+
| package | 762     |
| status  | success |
+---------+---------+


# Acknowledgements
TCIA is funded by the [Cancer Imaging Program (CIP)](https://imaging.cancer.gov/), a part of the United States [National Cancer Institute (NCI)](https://www.cancer.gov/).  It is managed by the [Frederick National Laboratory for Cancer Research (FNLCR)](https://frederick.cancer.gov/) and hosted by the [University of Arkansas for Medical Sciences (UAMS)](https://www.uams.edu/)

This notebook was created by [Justin Kirby](https://www.linkedin.com/in/justinkirby82/), [Quasar Jarosz](https://www.linkedin.com/in/quasarjarosz/) and [Lawrence Tarbox](https://www.linkedin.com/in/lawrence-tarbox-088335/). If you leverage this notebook or any TCIA datasets in your work, please be sure to comply with the [TCIA Data Usage Policy](https://wiki.cancerimagingarchive.net/x/c4hF). In particular, make sure to cite the DOI(s) for the specific TCIA datasets you used in addition to the following paper!

## TCIA Citation

Clark, K., Vendt, B., Smith, K., Freymann, J., Kirby, J., Koppel, P., Moore, S., Phillips, S., Maffitt, D., Pringle, M., Tarbox, L., & Prior, F. (2013). The Cancer Imaging Archive (TCIA): Maintaining and Operating a Public Information Repository. Journal of Digital Imaging, 26(6), 1045–1057. https://doi.org/10.1007/s10278-013-9622-7